In [2]:
from __future__ import annotations

import os
import warnings
from pathlib import Path
import shutil
import subprocess
from typing import Optional
from dataclasses import dataclass, field

import pandas as pd

# from pymatgen.core import Structure
# from pymatgen.io.vasp.inputs import Incar, Kpoints
# from pymatgen.io.vasp.outputs import Vasprun

from ase import Atoms

from pyiron_workflow import Workflow

# from pyiron_atomistics.vasp.output import parse_vasp_output as pvo

# from pyiron_snippets.logger import logger
# from pyiron_snippets.resources import ResourceResolver

# from pyiron_nodes.atomistic.engine.lammps import Shell
# from pyiron_nodes.lammps import ShellOutput


ModuleNotFoundError: No module named 'pyiron_nodes'

In [4]:
import os
from pathlib import Path

def read_potcar_config(config_file: Path) -> dict:
    """
    Reads the POTCAR configuration from a file and resolves the paths dynamically based on config content.
    
    Args:
        config_file (Path): Path to the configuration file.
        
    Returns:
        dict: A dictionary containing the resolved paths and the default POTCAR path.
        
    Raises:
        ValueError: If no valid default_POTCAR_set is found in the config.
    """
    config_data = {}

    # Read the configuration file
    try:
        with open(config_file, "r") as f:
            for line in f:
                # Skip comments and empty lines
                line = line.strip()
                if not line or line.startswith("#"):
                    continue
                
                # Split the line into key and value
                if "=" in line:
                    key, value = line.split("=", 1)
                    key = key.strip()
                    value = value.strip()
                    
                    # Store the configuration
                    config_data[key] = value

        # Resolve the pyiron_vasp_resources path
        pyiron_vasp_resources = config_data.get("pyiron_vasp_resources", "")
        default_POTCAR_set = config_data.get("default_POTCAR_set")
        default_functional = config_data.get("default_functional")
        # Dynamically identify all POTCAR sets based on keys in the config file
        potcar_sets = []
        for key in config_data:
            if key.startswith("vasp_POTCAR_path_"):
                potcar_sets.append(key.split("vasp_POTCAR_path_")[1])

        # Check if a valid default_POTCAR_set is provided
        if not default_POTCAR_set or default_POTCAR_set not in potcar_sets:
            raise ValueError(f"Unknown or missing default_POTCAR_set: {default_POTCAR_set}. Valid options: {potcar_sets}")
        
        # Check if a valid default_functional is provided
        if not default_functional or default_functional not in potcar_sets:
            raise ValueError(f"Unknown or missing default_functional: {default_functional}. Valid options: GGA, LDA")
        
        # Dynamically generate the paths for all potpaw sets based on the config content
        for potcar_set in potcar_sets:
            key = f"vasp_POTCAR_path_{potcar_set}"
            config_data[key] = os.path.join(pyiron_vasp_resources, potcar_set)

        # Set the default POTCAR path
        config_data["default_POTCAR_path"] = config_data[f"vasp_POTCAR_path_{default_POTCAR_set}"]

        return config_data

    except FileNotFoundError:
        print(f"Configuration file not found: {config_file}")
        return {}

    except Exception as e:
        print(f"Error reading configuration file: {e}")
        return {}

# Example usage
config_file = Path("/root/github_dev/github_pyiron/pyiron_nodes/pyiron_nodes/atomistic/engine/example_pyiron_vasp_config")
potcar_config = read_potcar_config(config_file)

# Print the resolved paths
for key, path in potcar_config.items():
    print(f"{key}: {path}")


default_POTCAR_set: potpaw64
default_functional: GGA
pyiron_vasp_resources: /home/pyiron_resources_cmmc/vasp
vasp_POTCAR_path_potpaw64: /home/pyiron_resources_cmmc/vasp/potpaw64
vasp_POTCAR_path_potpaw54: /home/pyiron_resources_cmmc/vasp/potpaw54
vasp_POTCAR_path_potpaw52: /home/pyiron_resources_cmmc/vasp/potpaw52
default_POTCAR_path: /home/pyiron_resources_cmmc/vasp/potpaw64


In [6]:
POTCAR_library_path

PosixPath('/path/to/your/vasp/potentials')

In [ ]:
resource_plugin_name = "lammps"
ResourceResolver(
            state.settings.resource_paths,
            cls.resource_plugin_name,
            "potentials",
        )